# Natural Language Pre-Processing

In [3]:
# Use one of these to install nltk if needed
# !pip install nltk
# !conda install -c anaconda nltk

In [4]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Use this to download the stopwords if you haven't already - only ever needs to be run once

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shelleywang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Learning Goals

- Describe the basic concepts of NLP
- Use pre-processing methods for NLP
    - Tokenization
    - Stopwords removal

# Overview of NLP

NLP allows computers to interact with text data in a structured and sensible way. In short, we will be breaking up series of texts into individual words (or groups of words), and isolating the words with **semantic value**.  We will then compare texts with similar distributions of these words, and group them together.

In this section, we will discuss some steps and approaches to common text data analytic procedures. Some of the applications of natural language processing are:
- Chatbots 
- Speech recognition and audio processing 
- Classifying documents 

Here is an example that uses some of the tools we use in this notebook.  
  -[chicago_justice classifier](https://github.com/chicago-justice-project/article-tagging/blob/master/lib/notebooks/bag-of-words-count-stemmed-binary.ipynb)

We will introduce you to the preprocessing steps, feature engineering, and other steps you need to take in order to format text data for machine learning tasks. 

We will also introduce you to [**NLTK**](https://www.nltk.org/) (Natural Language Toolkit), which will be our main tool for engaging with textual data.

<img src="images/nlp_process.png" style="width:1000px;">

# Preprocessing for NLP

The goal when pre-processing text data for NLP is to remove as many unnecessary words as possible while preserving as much semantic meaning as possible. This will improve your model performance dramatically.

You can think of this sort of like dimensionality reduction. The unique words in your corpus form a **vocabulary**, and each word in your vocabulary is essentially another feature in your model. So we want to get rid of unnecessary words and consolidate words that have similar meanings.

We will be working with a dataset which includes both satirical** (The Onion) and real news (Reuters) articles. We refer to the entire set of articles as the **corpus**.  

![the_onion](images/the_onion.jpeg) ![reuters](images/reuters.png)

In [6]:
corpus = pd.read_csv('data/satire_nosatire.csv')
corpus.shape

(1000, 2)

In [7]:
corpus.tail()

,body,target
995,Britain’s opposition leader Jeremy Corbyn wou...,0
996,Turkey will take over the fight against Islam...,0
997,Malaysia is seeking $7.5 billion in reparatio...,0
998,An Israeli court sentenced a Palestinian to 1...,0
999,At least 22 people have died due to landslide...,0


Our goal is to detect satire, so our target class of 1 is associated with The Onion articles.  

In [8]:
corpus.loc[10].body

'In recognition of the brave and altruistic Americans who risk their health and safety for the greater good, Pentagon officials announced Thursday that the U.S. military would honor the sacrifices of NFL players by wearing their jerseys throughout December. “Every week, these men are out there on the gridiron, putting their bodies on the line for our country. Having our soldiers wear these jerseys and caps is the least we can do to show our support,” said Pentagon spokesperson Robert D’Amato, who announced that all active-duty soldiers would be sporting gear from all 32 teams to raise awareness of the people who put their lives aside to preserve the American way of life. “These men leave their homes and their families and travel to cities across the country to uphold our nation’s traditions. They are battered and bruised for years, their lives often cut short while we sit back in our barracks and enjoy our freedom. Some of them are never the same after they end their service, and yet w

In [9]:
corpus.loc[10].target

1

In [10]:
corpus.loc[502].body

' Indonesia will launch a renewed search effort as early as Tuesday to find the cockpit voice recorder from a Lion Air jet that crashed into the Java Sea in October, the head of its accident investigation agency said. “If the weather is good, the ship will start to depart today,” National Transporation Safety Commission (KNKT) Chief Soerjanto Tjahjono told Reuters on Tuesday. The crash, the world’s first of a Boeing Co (BA.N) 737 MAX jet and the deadliest of 2018, killed all 189 people on board. Investigators last week said they planned to use a navy ship for a fresh search for the crashed jet’s second “black box” after a 10-day effort funded by Lion Air failed to find the cockpit voice recorder (CVR). A KNKT source, speaking on condition of anonymity, told Reuters the team will have seven days using the ship KRI Spica to find the CVR, which could hold vital clues giving investigators insight into the actions of the doomed jet’s pilots. Tjahjono declined to comment on whether there was

In [11]:
corpus.loc[502].target

0

Each article in the corpus is refered to as a **document**.

It is a balanced dataset with 500 documents of each category. 

In [12]:
corpus.target.value_counts()

1    500
0    500
Name: target, dtype: int64

**Discussion:** Let's think about the use cases of being able to correctly separate satirical from authentic news. What might be a real-world use case?  

In [13]:
# Thoughts here



## Tokenization 

In order to convert the texts into data suitable for machine learning, we need to break down the documents into smaller parts. 

The first step in doing that is **tokenization**.

Tokenization is the process of splitting documents into units of observations. We usually represent the tokens as __n-grams__, where n represent the number of consecutive words occuring in a document that we will consider a unit. In the case of unigrams (one-word tokens), the sentence "David works here" would be tokenized into:

- "David", "works", "here";

If we want (also) to consider bigrams, we would (also) consider:

- "David works" and "works here".

Let's consider a particular document in our corpus:

In [14]:
sample_document = corpus.iloc[1].body

In [15]:
sample_document

'Desperate to unwind after months of nonstop work investigating Russian influence in the 2016 election, visibly exhausted Special Counsel Robert Mueller powered his phone down Friday in order to give himself a break from any news concerning the probe over the holiday break. “The last thing I want when I’m spending time with my family is a cascade of push notifications telling me yet another Russian oligarch, political operative, or highly placed socialite used Deutsche Bank channels to funnel money to the campaign,” said the former FBI director, firmly holding down his phone’s power button and adding that he wants to be “completely present in the moment” while celebrating with his loved ones, not ruminating about who met with which diplomat or whether someone was using social media to tamper with his witnesses. “I just want to have two calm weeks where I don’t even think about Individual One. I won’t even say his name. I’ll have to wait to hear about any important developments in Janua

There are many ways to tokenize our document. 

It is a long string, so the first way we might consider is to split it by spaces.

**Knowledge Check:** How would we split our documents into words using spaces?

<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

    sample_document.split(' ')
    
</details>

In [16]:
# code


But this is not ideal. We are trying to create a set of tokens with **high semantic value**.  In other words, we want to isolate text which best represents the meaning in each document.

# Text Cleaning

Most NL Pre-Processing will include the following tasks:

  1. Remove capitalization  
  2. Remove punctuation  
  3. Remove stopwords  
  4. Remove numerals

We could manually perform all of these tasks with string operations.

## Capitalization

When we create our matrix of words associated with our corpus, **capital letters** will mess things up.  The semantic value of a word used at the beginning of a sentence is the same as that same word in the middle of the sentence.  In the two sentences:

sentence_one =  "Excessive gerrymandering in small counties suppresses turnout."   
sentence_two =  "Turnout is suppressed in small counties by excessive gerrymandering."  

'excessive' has the same semantic value, but will be treated as different tokens because of capitals.

In [17]:
sentence_one =  "Excessive gerrymandering in small counties suppresses turnout." 
sentence_two =  "Turnout is suppressed in small counties by excessive gerrymandering."

Excessive = sentence_one.split(' ')[0]
excessive = sentence_two.split(' ')[-2]
print(excessive, Excessive)
excessive == Excessive

excessive Excessive


False

In [18]:
manual_cleanup = None

In [19]:
print(f"Our initial token set for our sample document is {len(manual_cleanup)} words long")

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(f"Our initial token set for our sample document has \
{len(set(sample_document.split(' ')))} unique words")

In [ ]:
print(f"After removing capitals, our sample document has \
{len(set(manual_cleanup))} unique words")

## Punctuation

Like capitals, splitting on white space will create tokens which include punctuation that will muck up our semantics.  

Returning to the above example, 'gerrymandering' and 'gerrymandering.' will be treated as different tokens.

In [ ]:
no_punct = sentence_one.split(' ')[1]
punct = sentence_two.split(' ')[-1]
print(no_punct, punct)
no_punct == punct

In [ ]:
## Manual removal of punctuation

None

In [ ]:
manual_cleanup = None

In [ ]:
print(f"After removing punctuation, our sample document has \
{len(set(manual_cleanup))} unique words")

In [ ]:
manual_cleanup[:20]

## Stopwords

Stopwords are the **filler** words in a language: prepositions, articles, conjunctions. They have low semantic value, and often need to be removed.  

Luckily, NLTK has lists of stopwords ready for our use.

In [ ]:
None

In [ ]:
None

Let's see which stopwords are present in our sample document.

In [ ]:
stops = None

In [ ]:
print(f'There are {len(stops)} instances of {len(set(stops))} \
stopwords in the sample document')

In [ ]:
print(f'The {len(stops)} instances make up \
{len(stops)/len(manual_cleanup): 0.2%} of our text')

Let's also use the **FreqDist** tool to look at the makeup of our text before and after removal:

In [ ]:
fdist = FreqDist(manual_cleanup)
plt.figure(figsize=(10, 10))
fdist.plot(30);

In [ ]:
manual_cleanup = None

In [ ]:
sample_document

In [ ]:
manual_cleanup[:10]

In [ ]:
# We can also customize our stopwords list

None

In [ ]:
print(f'After removing stopwords, there are {len(set(manual_cleanup))} unique words left')

In [ ]:
fdist = FreqDist(manual_cleanup)
plt.figure(figsize=(10, 10))
fdist.plot(30);

#### Numerals

Numerals also usually have low semantic value. Their removal can help improve our models. 

In [ ]:
manual_cleanup = None

In [ ]:
# drop empty strings

manual_cleanup = None

In [ ]:
print(f'After removing numerals, there are {len(set(manual_cleanup))} unique words left')

# Regex

Regex allows us to match strings based on a pattern.  This pattern comes from a language of identifiers, which we can begin exploring on the cheatsheet found here:
  -   https://regexr.com/

A few key symbols:
  - . : matches any character
  - \d, \w, \s : represent digit, word, whitespace  
  - *, ?, +: matches 0 or more, 0 or 1, 1 or more of the preceding character  
  - [A-Z]: matches any capital letter  
  - [a-z]: matches lowercase letter  

Other helpful resources:
  - https://regexcrossword.com/
  - https://www.regular-expressions.info/tutorial.html

We can use regex to isolate numerals:

In [ ]:
sample_document

In [ ]:
pattern = None
number = None
number

In [ ]:
pattern2 = None
number2 = re.findall(pattern2, sample_document)
number2

## `RegexpTokenizer()`

Sklearn and NLTK provide us with a suite of **tokenizers** for our text preprocessing convenience. So we don't have to do this all by hand every time!

In [ ]:
sample_document

In [ ]:
# Remember that the '?' indicates 0 or 1!

re.findall(r"([a-zA-Z]+(?:'[a-z]+)?)", "I'd")

In [ ]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = None
sample_doc = None

In [ ]:
sample_doc = [token.lower() for token in sample_doc]
sample_doc = [token for token in sample_doc if token not in sw]

In [ ]:
sample_document

In [ ]:
sample_doc[:10]

In [ ]:
print(f'We are down to {len(set(sample_doc))} unique words')

# Exercise: NL Pre-Processing

**Activity:** Use what you've learned to preprocess the fourth article. How does the length and number of unique words in the article change?

<p>
</p>
<details>
    <summary><b><u>Click Here for Answer Code</u></b></summary>

    fourth_document = corpus.iloc[3].body
    print(f'We start with {len(fourth_document.split())} words')
    print(f'We start with {len(set(fourth_document.split()))} unique words')
    
    fourth_doc = tokenizer.tokenize(fourth_document)
    fourth_doc = [token.lower() for token in fourth_doc]
    fourth_doc = [token for token in fourth_doc if token not in sw]
    print(f'We end with {len(fourth_doc)} words')
    print(f'We end with {len(set(fourth_doc))} unique words')

    
</details>

In [ ]:
## Your code here
